In [96]:
import pandas as pd
import nltk 
from nltk.corpus import stopwords
from nltk.tokenize import ToktokTokenizer
import json
import gzip

In [97]:
def parse(path):
  g = gzip.open(path, 'rb')
  for l in g:
    yield json.loads(l)

def getDF(path):
  i = 0
  df = {}
  for d in parse(path):
    df[i] = d
    i += 1
  return pd.DataFrame.from_dict(df, orient='index')

df = getDF('Magazine_Subscriptions.json.gz')

In [98]:

# Data cleanup

# lowercase review text
df["reviewText"] = df["reviewText"].str.lower()
# remove unverified
df = df[df["verified"] == True]
# remove unixReviewTime
df.drop(["unixReviewTime","reviewerID", "image", "style", "asin"], axis=1, inplace=True)

# set votes containing NaN to 0
df["vote"] = df["vote"].fillna(0)
# overall to int from float
df["overall"] = df["overall"].astype(int)
# remove NaN reviewText
df["reviewText"] = df["reviewText"].fillna("")

# remove stop words
tokenizer = ToktokTokenizer()
stop_words = stopwords.words("english")
df["reviewText"] = df["reviewText"].apply(lambda x: ' '.join([word for word in x.split() if word not in stop_words]))

# remove punctuation from reviewText
# [^\w\s]' -> looks for anything that isnt a word or whitespace to remove
df["reviewText"] = df["reviewText"].str.replace('[^\w\s]',"")

# MISSING:
# - stemming
# - lemmatization

/tmp/ipykernel_51296/447077774.py:23: FutureWarning: The default value of regex will change from True to False in a future version.
  df["reviewText"] = df["reviewText"].str.replace('[^\w\s]',"")


4                                   great read every issue
5        magazine great times technology magazines new ...
6        ive read maximum pc mpc many years articles te...
7        ordered magazine grandson then 7 going 30 wasi...
8        subscribed nook version years thoroughly enjoy...
                               ...                        
89684    nice surprise boyfriend really enjoyed olympic...
89685    magazine looks like printed recycled shoe rubb...
89686                cant go wrong si subscription pleased
89687    magazine far time favorite one metal detecting...
89688                     nice magazine good info articles
Name: reviewText, Length: 58654, dtype: object